In [ ]:
from google.colab import drive   
drive.mount('/content/drive')

Mounted at /content/drive


# Tiền xử lí dữ liệu VnExpress thu thập được

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# Tiền xử lí file dữ liệu thu thập từ vnexpress
data = open('/content/drive/My Drive/code_crawl/vnexpress/raw.txt', 'r', encoding='utf8').readlines()
for i in range(len(data)):
  data[i] = BeautifulSoup(data[i], "lxml").text
  data[i] = data[i].replace('\xa0', ' ')
  

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://vietnamese.vietnam.usembassy.gov/yseali-2016.html
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://gomhailong.vn/am-sac-thuoc-dien-tu-dong
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://gomhailong.vn
" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that docu

In [ ]:
# loại bỏ những câu quá ngắn
data1 = []
for i in range(len(data)):
  if len(data[i].split()) > 10:
    data1.append(data[i])
data = data1
data1 = []

In [ ]:
import re
# tiền xử lí các số, email, website
for i in range(len(data)):
  # chuyển các con số thành NUMBER
  data[i] = re.sub("\d+", "NUMBER", data[i])
  # chuyển các địa chỉ email thành EMAIL
  data[i] = re.sub("/\S+@\S+\.\S+/", "EMAIL", data[i])
  # chuyển các địa chỉ website thành URL
  data[i] = re.sub("((\w+:\/\/\S+)|(\w+[\.:]\w+\S+))[^\s,\.]", "URL", data[i])

In [ ]:
# Sau khi loại thay thế các địa chỉ mail, website,... ta tiến hành xử lí các dấu câu và khoảng trắng dư thừa
data1 = []
for i in data:
  temp = i.split('.')
  for j in temp:
    if len(j.split()) >8:
      temp1 = re.sub('''[,#!\^&\*;:{}=`?\[\]'"“”‘’~()]''', " ", j)
      temp1 = re.sub("\s+", " ", temp1)
      temp1 = temp1.strip()
      temp1 = temp1.lower()
      data1.append(temp1)
data = data1.copy()

In [ ]:
# ghi lại tập dữ liệu sau khi đã được tiền xử lí
f = open('/content/drive/My Drive/code_crawl/vnexpress/text_preprocessor1.txt', 'w', encoding='utf8')
for i in data:
  f.write(i+'\n')
f.close()

# Tạo model với tập dữ liệu từ vnexpress đã tiền xử lí

## Mô hình cbow

In [ ]:
# tạo mô hình với dữ liệu bình thường
from gensim.models import Word2Vec
data_input = '/content/drive/My Drive/code_crawl/vnexpress/text_preprocessor1.txt'
model = Word2Vec(corpus_file=data_input, vector_size=300, window=4, min_count=5, sg=0)

In [ ]:
# lưu mô hình cbow lại
import pickle
pickle.dump(model, open('/content/drive/My Drive/Colab Notebooks/word2vec/cbow/model.bin', 'wb'))

## Mô hình skip-gram

In [ ]:
# tạo mô hình với dữ liệu bình thường
from gensim.models import Word2Vec
data_input = '/content/drive/My Drive/code_crawl/vnexpress/text_preprocessor1.txt'
model = Word2Vec(corpus_file=data_input, vector_size=300, window=4, min_count=5, sg=1)

In [ ]:
# lưu mô hình cbow lại
import pickle
pickle.dump(model, open('/content/drive/My Drive/Colab Notebooks/word2vec/skip-gram/model.bin', 'wb'))

# Tạo tập corpus, small corpus, badword phục vụ sửa lỗi chính tả và tạo dữ liệu train cho mô hình
- Phần này chỉ thao tác trên phần dữ liệu train của HateSpeech
- Corpus là tập từ vựng được tạo từ bộ từ vựng của vnexpress + từ vựng bộ train
- Small corpus là tập từ vựng bộ train
- Badword ở đây bao gồm những từ xuất hiện nhiều trong lớp hate

In [ ]:
import numpy as np
import pickle

In [ ]:
dir_folder = '/content/drive/My Drive/Colab Notebooks/dump_hatespeech/'
def load(file_name, dir_folder):
  f = open(dir_folder + file_name, 'rb')
  variable = pickle.load(f)
  f.close()
  return variable

In [ ]:
# đọc dữ liệu từ phần train của hate speech
train_x = load('train_x.dump', dir_folder)
train_y = load('train_y.dump' , dir_folder)

In [ ]:
print('X length: ',len(train_x))
print('y length:', len(train_y))

X length:  16276
y length: 16276


Xử lí dữ liệu tập train_x
- Thay thế các kí tự giống nhau lặp lại nhiều lần
- Thay thế đoạn html còn sót lại \x01
- Thay thế dấu gạch và các khoảng trắng dư thừa


In [ ]:
import re 
train_x = list(train_x)
temp = []
for index, i in enumerate(train_x):
  i = re.sub(r'(.)\1{2,}', r'\1\1', i)
  i = re.sub('\x01', '', i)
  i = re.sub('_|\s+', ' ', i)
  train_x[index] = i

## Thống kê tần suất của từng lớp

In [ ]:
# hàm thống kê tần suất kí tự
from collections import Counter
def word_statistics(sent_list):
  """
  input: 
    sent_list: list include sents
  ouput: 
    words: list include words and freq
    log: text include lines, each line is <word> <freq>
  """
  data = ' '.join(sent_list)
  c = Counter(data.split())
  words = []
  for i in list(c):
    words.append([i, c[i]])
  def freq(w):
    return w[1]
  # sort danh sách từ lại
  words.sort(key=freq, reverse=True)
  # tạo thành 1 file văn bản thống kê tần suất các từ
  log = ''
  for i in words:
    log = log + i[0] + ' ' + str(i[1]) + '\n'
  return words, log

In [ ]:
'|'.join('àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ')

'à|á|ã|ạ|ả|ă|ắ|ằ|ẳ|ẵ|ặ|â|ấ|ầ|ẩ|ẫ|ậ|è|é|ẹ|ẻ|ẽ|ê|ề|ế|ể|ễ|ệ|đ|ì|í|ĩ|ỉ|ị|ò|ó|õ|ọ|ỏ|ô|ố|ồ|ổ|ỗ|ộ|ơ|ớ|ờ|ở|ỡ|ợ|ù|ú|ũ|ụ|ủ|ư|ứ|ừ|ử|ữ|ự|ỳ|ỵ|ỷ|ỹ|ý|À|Á|Ã|Ạ|Ả|Ă|Ắ|Ằ|Ẳ|Ẵ|Ặ|Â|Ấ|Ầ|Ẩ|Ẫ|Ậ|È|É|Ẹ|Ẻ|Ẽ|Ê|Ề|Ế|Ể|Ễ|Ệ|Đ|Ì|Í|Ĩ|Ỉ|Ị|Ò|Ó|Õ|Ọ|Ỏ|Ô|Ố|Ồ|Ổ|Ỗ|Ộ|Ơ|Ớ|Ờ|Ở|Ỡ|Ợ|Ù|Ú|Ũ|Ụ|Ủ|Ư|Ứ|Ừ|Ử|Ữ|Ự|Ỳ|Ỵ|Ỷ|Ỹ|Ý'

In [ ]:
import re
# tách riêng các câu của từng lớp ra
clean_sents = []
offensive_sents = []
hate_sents = []
dau = 'à|á|ã|ạ|ả|ă|ắ|ằ|ẳ|ẵ|ặ|â|ấ|ầ|ẩ|ẫ|ậ|è|é|ẹ|ẻ|ẽ|ê|ề|ế|ể|ễ|ệ|đ|ì|í|ĩ|ỉ|ị|ò|ó|õ|ọ|ỏ|ô|ố|ồ|ổ|ỗ|ộ|ơ|ớ|ờ|ở|ỡ|ợ|ù|ú|ũ|ụ|ủ|ư|ứ|ừ|ử|ữ|ự|ỳ|ỵ|ỷ|ỹ|ý|À|Á|Ã|Ạ|Ả|Ă|Ắ|Ằ|Ẳ|Ẵ|Ặ|Â|Ấ|Ầ|Ẩ|Ẫ|Ậ|È|É|Ẹ|Ẻ|Ẽ|Ê|Ề|Ế|Ể|Ễ|Ệ|Đ|Ì|Í|Ĩ|Ỉ|Ị|Ò|Ó|Õ|Ọ|Ỏ|Ô|Ố|Ồ|Ổ|Ỗ|Ộ|Ơ|Ớ|Ờ|Ở|Ỡ|Ợ|Ù|Ú|Ũ|Ụ|Ủ|Ư|Ứ|Ừ|Ử|Ữ|Ự|Ỳ|Ỵ|Ỷ|Ỹ|Ý'
for index, i in enumerate(train_y):
  n = 0
  for j in train_x[index].split():
    if re.search(dau, j):
      n += 1
  if n < len(train_x[index].split())/3:
    continue

  if i[0] == 1:
    clean_sents.append(train_x[index])
  elif i[1] == 1:
    offensive_sents.append(train_x[index])
  else:
    hate_sents.append(train_x[index])

print("Clean length    \t", len(clean_sents))
print("Offensive length\t", len(offensive_sents))
print("Hate length     \t", len(hate_sents))

Clean length    	 13672
Offensive length	 751
Hate length     	 526


In [ ]:
# Thống kê lần lượt 3 lớp clean, offensive và hate
log_folder_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/temp/word_statistics/'
# clean
clean_words, clean_log = word_statistics(clean_sents)
open(log_folder_dir + 'clean_words.txt', 'w').write(clean_log)
# offensive
offensive_words, offensive_log = word_statistics(offensive_sents)
open(log_folder_dir + 'offensive_words.txt', 'w').write(offensive_log)
# hate
hate_words, hate_log = word_statistics(hate_sents)
open(log_folder_dir + 'hate_words.txt', 'w').write(hate_log)
print("Finish statistics.")
print("Clean words length    \t", len(clean_words))
print("Offensive words length\t", len(offensive_words))
print("Hate words length     \t", len(hate_words))

Finish statistics.
Clean words length    	 12027
Offensive words length	 1953
Hate words length     	 2179


Lấy ngưỡng từng bộ từ vựng
- clean > 8
- hate & offisive > 2

In [ ]:
def cut_threshold(words, threshold):
  """
  input:
    words: (list) contain objects. each object is [word, freq]
    threshold: (int) cut at the position where the frequency is greater than the threshold
  """
  for index, i in enumerate(words):
    if not (i[1] > threshold):
      return words[:index]
  print("Error did not return the list")
  return None

#clean
clean_words = cut_threshold(clean_words, 8)
# offensive
offensive_words = cut_threshold(offensive_words, 2)
# hate
hate_words = cut_threshold(hate_words, 2)

print("Finish cut threshold")
print("Clean words length    \t", len(clean_words))
print("Offensive words length\t", len(offensive_words))
print("Hate words length     \t", len(hate_words))

Finish cut threshold
Clean words length    	 2945
Offensive words length	 645
Hate words length     	 822


## Tạo tập small corpus

Gộp các bộ từ vựng lại với nhau

In [ ]:
def merge_vocab(*vocab_list, weight = None, isFreq = True):
  if isFreq:
    if weight != None:
      result = dict(vocab_list[0])
      for i in result:
        result[i] = result[i]*weight[0]

      for index, i in list(enumerate(vocab_list))[1:]:
        i = dict(i)
        for j in i:
          try:
            result[j] = result[j] + i[j]*weight[index]
          except:
            result[j] = i[j]*weight[index]
      return result
  else:
    if weight == None:
      result = set(vocab_list[0])
      for i in vocab_list[1:]:
        result = result | set(i)
      return result
  print("Error: incomplete function")

In [ ]:
small_corpus = merge_vocab(clean_words, hate_words, offensive_words, weight=[1,12,6])

In [ ]:
# sort lại danh sách
small_corpus = sorted(small_corpus.items(), key=lambda x: x[1], reverse=True)

In [ ]:
# ghi file small corpus xuống
small_corpus_folder_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/'
s = ''
for i in small_corpus:
  # loại bỏ ác từ 1 kí tự hoặc thuộc 1 trong 11 loại phụ âm ghép
  if len(i[0]) != 1 and i[0] not in ['ph', 'th', 'tr', 'gi', 'ch', 'nh', 'ng', 'kh', 'gh', 'ngh', 'qu']:
    s = s + i[0] + ' ' + str(i[1]) + '\n'
s = s[:-1]
open(small_corpus_folder_dir + 'small_corpus.txt', 'w').write(s)

16282

## Tạo tập corpus refine

In [ ]:
small_corpus = dict(small_corpus)
# đọc tập từ vựng của vnexpres
def read_vocab_vne(dir_vocab_vnexpress, threshold):
  """
  Input:
    dir_vocab_vnexpress: the path of the vnexpress vocabulary file
    threshold: (int) cut at the position where the frequency is greater than the threshold
  Output:
    (set) vocabulary
  """
  temp = open(dir_vocab_vnexpress).read().split('\n')[:-1]
  vocab = []
  for i in temp:
    word, freq = i.split()
    if int(freq) > threshold:
      vocab.append(word)
  return set(vocab)

dir_vocab_vnexpress = '/content/drive/My Drive/code_crawl/vnexpress/vocab1.txt'
corpus = read_vocab_vne(dir_vocab_vnexpress, 100)
print("Vocabulary Vnexpress length: ", len(corpus))

# merge corpus and corpus refine
corpus = corpus | set(small_corpus)
print("Corpus length:               ", len(corpus))

Vocabulary Vnexpress length:  11815
Corpus length:                12210


In [ ]:
# merge thêm tập từ vựng tokenizer
py_words = open('/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/pyvi_words_edited.txt').read().split('\n')
corpus = corpus | set(py_words)

In [ ]:
# ghi file corpus xuống
dir_corpus_file = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/corpus.txt'
open(dir_corpus_file, 'w').write('\n'.join(corpus))

295971

Tạo tập vocab hate and offensive

In [ ]:
hate_offensive_vocab = merge_vocab(hate_words, offensive_words, weight=[1,1])
# sort lại danh sách
hate_offensive_vocab = sorted(hate_offensive_vocab.items(), key=lambda x: x[1], reverse=True)
# ghi file small corpus xuống
hate_offensive_vocab_dir = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/'
open(hate_offensive_vocab_dir + 'hate_offensive_vocab1.txt', 'w').write('\n'.join(dict(hate_offensive_vocab)))

4169

## Tạo dữ liệu train

In [ ]:
# thống kê tần suất các lớp
print("Clean sent length    \t", len(clean_sents))
print("Offensive sent length\t", len(offensive_sents))
print("Hate sent length     \t", len(hate_sents))

Clean sent length    	 13672
Offensive sent length	 751
Hate sent length     	 526


In [ ]:
import random
# tạo dữ liệu train
data = clean_sents + hate_sents*12 + offensive_sents*12
random.shuffle(data)

In [ ]:
# ghi xuôgns thành file
open('/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/fasttext_data/train.txt', 'w').write('\n'.join(data))

2414481

# Turning lại dữ liệu theo miền hate speech

## CBOW

In [ ]:
import pickle
from gensim.models import Word2Vec
# turning lại mô hình
data_input = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/fasttext_data/train.txt'
model = pickle.load(open('/content/drive/My Drive/Colab Notebooks/word2vec/cbow/model.bin', 'rb'))
model.build_vocab(corpus_file=data_input, update=True)
model.train(corpus_file=data_input, word_count=5, epochs=10, total_words=model.corpus_count)

In [ ]:
# lấy tập từ vựng của mô hình
# fasttext_vocab = model.get_words()
fasttext_vocab = list(model.wv.vocab)
# giao với tập corpus
fasttext_vocab = set(fasttext_vocab) & corpus
print("Finish get vocabulary from fasttext and merge corpus.")
print("Corpus length            ", len(corpus))
print("Fasttext corpus lenght   ", len(fasttext_vocab))

In [ ]:
# lấy tập vector ứng với tập từ vựng trên
vector = []
for word in fasttext_vocab:
  vector.append(model.wv.get_vector(word))
  # vector.append(model.get_word_vector(word))
print("Finish get vector from fasttext.")
print("vector length   ", len(fasttext_vocab))

Ghi dữ liệu xuống

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/Model/cbow/'
# ghi file vocab
open(folder_path + 'vocab.txt', 'w').write('\n'.join(fasttext_vocab))
# ghi file vector
vec = np.array(vector)
np.save(folder_path + 'vec.npy', vec)
print('Finish write vocabulary and vector.')
print('Vector shape: ',vec.shape)

## SKIP-GRAM

In [ ]:
import pickle
from gensim.models import Word2Vec
# turning lại mô hình
data_input = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/fasttext_data/train.txt'
model = pickle.load(open('/content/drive/My Drive/Colab Notebooks/word2vec/skip-gram/model.bin', 'rb'))
model.build_vocab(corpus_file=data_input, update=True)
model.train(corpus_file=data_input, word_count=5, epochs=10, total_words=model.corpus_count)

In [ ]:
# lấy tập từ vựng của mô hình
# fasttext_vocab = model.get_words()
fasttext_vocab = list(model.wv.vocab)
# giao với tập corpus
fasttext_vocab = set(fasttext_vocab) & corpus
print("Finish get vocabulary from fasttext and merge corpus.")
print("Corpus length            ", len(corpus))
print("Fasttext corpus lenght   ", len(fasttext_vocab))

In [ ]:
# lấy tập vector ứng với tập từ vựng trên
vector = []
for word in fasttext_vocab:
  vector.append(model.wv.get_vector(word))
  # vector.append(model.get_word_vector(word))
print("Finish get vector from fasttext.")
print("vector length   ", len(fasttext_vocab))

Ghi dữ liệu xuống

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/Model/skip-gram/'
# ghi file vocab
open(folder_path + 'vocab.txt', 'w').write('\n'.join(fasttext_vocab))
# ghi file vector
vec = np.array(vector)
np.save(folder_path + 'vec.npy', vec)
print('Finish write vocabulary and vector.')
print('Vector shape: ',vec.shape)

## Fasttext

In [ ]:
!pip install fasttext
import fasttext
pre_model = '/content/drive/My Drive/Colab Notebooks/Hate Speech Detection/Data/cc.vi.300.vec'
# train mô hình
data_input = '/content/drive/My Drive/Colab Notebooks/KLTN_hatespeech/fasttext_data/train.txt'
model = fasttext.train_supervised(input=data_input, epoch=10, lr = 0.1, dim=300, minCount=5, pretrainedVectors= pre_model )

In [ ]:
# lấy tập từ vựng của mô hình
fasttext_vocab = model.get_words()
# fasttext_vocab = list(model.wv.vocab)
# giao với tập corpus
fasttext_vocab = set(fasttext_vocab) & corpus
print("Finish get vocabulary from fasttext and merge corpus.")
print("Corpus length            ", len(corpus))
print("Fasttext corpus lenght   ", len(fasttext_vocab))

In [ ]:
# lấy tập vector ứng với tập từ vựng trên
vector = []
for word in fasttext_vocab:
  # vector.append(model.wv.get_vector(word))
  vector.append(model.get_word_vector(word))
print("Finish get vector from fasttext.")
print("vector length   ", len(fasttext_vocab))

Ghi dữ liệu xuống

In [ ]:
folder_path = '/content/drive/MyDrive/Colab Notebooks/KLTN_hatespeech/Model/fasttextfb/'
# ghi file vocab
open(folder_path + 'vocab.txt', 'w').write('\n'.join(fasttext_vocab))
# ghi file vector
vec = np.array(vector)
np.save(folder_path + 'vec.npy', vec)
print('Finish write vocabulary and vector.')
print('Vector shape: ',vec.shape)